# Predicción



La prediccion se realiza sobre las imagenes preprocesadas en el primer notebook. 

### Sentinel 2 rgb nir swir diffsw

In [1]:
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import os

2022-02-07 17:44:27.305431: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/miniconda3/envs/tf-gpu/lib/python3.7/site-packages/cv2/../../lib64:
2022-02-07 17:44:27.305462: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
#!rm -rf /home/ro/basurales/data/data_results/
#!ls /home/ro/basurales/data/modelo/weights

UNet_basurales_6D_160_30_spe150.h5


In [10]:
# modelo ya entrenado
#!gsutil cp -r gs://dym-datasets-public/basurales/weights/*FINAL* ../data/modelo/weights/

Copying gs://dym-datasets-public/basurales/weights/UNet_basurales_6D_100_30_spe150_FINAL.h5...
\ [1 files][356.2 MiB/356.2 MiB]                                                
Operation completed over 1 objects/356.2 MiB.                                    


Definimos el la configuracion para la predicción. Debemos pasar la ruta a las imagenes del dataset de prediccion y al modelo.

In [11]:

predict_config = PredictConfig(
                                    images_path=os.path.join('../dataset/data_predict/','6D','100_100'), # ruta a las imagenes sobre las cuales queremos predecir
                                    results_path=os.path.join('../dataset/data_results/','6D' ,'100_100'), # ruta de destino para nuestra predicción
                                    batch_size=16,
                                    model_path=os.path.join('../data/modelo/weights/', 'UNet_basurales_6D_100_30_spe150_FINAL.h5'),  #  ruta al modelo (.h5)
                                    height=160,
                                    width=160,
                                    n_channels=6,
                                    n_classes=1,
                                    class_weights= [1])
          
   

In [12]:
predict(predict_config)  # Ejecuta la predicción

Total images to predict (../dataset/data_predict/6D/100_100/images/*.tif): 660915
After skipping existing results: 660915


2022-02-07 17:49:19.397604: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-07 17:49:19.397649: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (quilmes-gpu-1): /proc/driver/nvidia/version does not exist
2022-02-07 17:49:19.397984: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  2%|▌                                   | 645/41308 [21:37<22:43:14,  2.01s/it]


KeyboardInterrupt: 

Podemos ver algunos resultados preliminales de la prediccion

In [89]:
import os
import random
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import tifffile as tiff
from skimage.transform import resize
from sklearn.preprocessing import minmax_scale

def plot_data_results_(
    num_samples: int = 3,
    fig_size=(20, 10),
    *,
    predict_config: PredictConfig,
    img_ch: int = 3,
    n_bands: int = 3
):
    """
    Plots some samples from the results directory.
    Parameters
    ----------
    num_samples : int
        Number of samples to plot.
    fig_size : tuple
        Figure size.
    img_ch : int
        Number of channels.
    predict_config : PredictConfig
        Prediction onfiguration object.
    """

    images = [
        os.path.basename(f)
        for f in sorted(glob(os.path.join(predict_config.results_path, "*.tif")))
    ]

    images = random.sample(images, num_samples)
    for img_file in images:
        try:
            if n_bands not in (1, 3):
                raise RuntimeError("n_bands option must be 1 or 3")
            
            fig, axes = plt.subplots(
                nrows=1, ncols=predict_config.n_classes + 1, figsize=(20, 40)
            )
                 
            if n_bands == 1:
          
                img_s2 = tiff.imread(
                    os.path.join(predict_config.images_path, "images", img_file)
                )[:, :, img_ch]
                axes[0].imshow(img_s2)
               
            if n_bands == 3:
             
                img_s2 = tiff.imread(
                    os.path.join(predict_config.images_path, "images", img_file)
                )[:, :, :3]
                axes[0].imshow(img_s2)
            
            # Prediccion
            mask_ = (
                tiff.imread(os.path.join(predict_config.results_path, img_file)) / 255
            )
   
            mask_ = resize(
                mask_,
                (predict_config.height, predict_config.width, predict_config.n_classes),
                mode="constant",
                preserve_range=True,
            )

          
           
            

            for c in range(predict_config.n_classes):
                axes[1 + c].imshow(np.squeeze(mask_[:, :, c]))

            plt.show()

        except Exception as err:
            print(err)

In [90]:
plot_data_results_(num_samples=10, fig_size=(5, 5), predict_config=predict_config, img_ch =3,n_bands =2)

RuntimeError: n_bands option must be 1 or 3